In [ ]:
# Pipeline 0 : lecture (data_ingestion)
# Pipeline 2 : processing ======================> to modify
# Pipeline 3 : model vs automl
# Pipepine 4 : hyperparameters
# Pipeline 5 : save best model + save pipeline


# + integrate metrics
# + integrate fairness 
# + integrate explainability
# + schedule end to end pipeline 
# + controle inference (best model + drift detector)

In [189]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pip:
  - azureml-defaults
  - keras
  - tensorflow<=2.4.*
  - numpy
  - scikit-learn
  - pandas
  - matplotlib

Overwriting conda_dependencies.yml


In [194]:
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace,RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl.utilities import get_primary_metrics
from azureml.data.datapath import DataPath
from azureml.core.datastore import Datastore
from azureml.pipeline.core import InputPortBinding
from azureml.core import Run
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter
from azureml.core import Dataset
from azureml.core import Environment

subscription_id = '---------'
resource_group = 'learning'
workspace_name = 'training_MLservices'

# Step 0 environnement

myenv = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

train_src = ScriptRunConfig(source_directory=script_folder,
                            script='model_script2.py',
                            compute_target=compute_target,
                            environment=myenv)
# Step 0 data ingestion
raw_ds = Dataset.get_by_name(ws, name='trainingdataset', version=1)
data_store = ws.get_default_datastore()
prepared_ds = OutputFileDatasetConfig(destination=(datastore, 'outputdataset/{run-id}')).register_on_complete(name='prepared_ds')



# Step 1 data preparation
step1 = PythonScriptStep(name = 'prepare data',
                         source_directory = 'scripts',
                         script_name = 'data_prep.py',
                         compute_target = 'jcharley2',
                         # Script arguments include PipelineData
                         arguments = ['--raw-ds', raw_ds.as_named_input('raw_data'),
                                      '--out_folder', prepared_ds])

# Step 2 Run Random Forest model
step2 = PythonScriptStep(name = 'train model',
                         source_directory = 'scripts',
                         script_name = 'model_script2.py',
                         compute_target = 'jcharley2',
                         # Pass as script argument
                         #arguments=[prepped_data.as_input(name='prepped_data')])
                        arguments=[prepared_ds.read_delimited_files().as_input(name='prepared_ds')],
                        runconfig=train_src.run_config)

# Step 2 bis (choice 2 or 2 bis) Run AutomML
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 4,
    "experiment_timeout_hours" : 0.10,
    "primary_metric" : 'Precision'
}

aml_run_config = RunConfiguration()
automl_config = AutoMLConfig(task = 'classification',
                             path = '.',
                             debug_log = 'automated_ml_errors.log',
                             compute_target = 'jcharley2',
                             run_configuration = aml_run_config,
                             featurization = 'auto',
                             training_data = [train_step_input],
                             label_column_name = 'EmployeeTargeted',
                             **automl_settings)
                             
# add to the pipeline
step2_bis = AutoMLStep(name='AutoML',
     automl_config=automl_config,
     passthru_automl_config=False,
     #outputs=[metrics_data,model_data],
     enable_default_model_output=False,
     enable_default_metrics_output=False,
     allow_reuse=True)


                   
pipeline = Pipeline(workspace=ws, steps=[step1,step2], description="test-pipeline_3")
#pipelinebis = Pipeline(workspace=ws, steps=[step1,step2_bis], description="test-pipeline_3bis")
pipeline.submit("test-pipeline", regenerate_outputs=True)


#published_pipeline1 = pipeline_run1.publish_pipeline(
#     name="Published_Titanic_Pipeline_Notebook",
#     description="Titanic_Pipeline_Notebook Published Pipeline Description",
#     version="1.0")

Created step prepare data [45245d52][aa48e885-e5c6-4619-a766-9ebe9997a5e9], (This step will run and generate new outputs)
Created step train model [6b3cdd91][3cc9f3ca-01f7-40b3-9562-f82fa5b2db4c], (This step will run and generate new outputs)
Submitted PipelineRun 49560045-af84-4b18-983c-e412ec75001a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/49560045-af84-4b18-983c-e412ec75001a?wsid=/subscriptions/a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b/resourcegroups/learning/workspaces/training_MLservices&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


Experiment,Id,Type,Status,Details Page,Docs Page
test-pipeline,49560045-af84-4b18-983c-e412ec75001a,azureml.PipelineRun,Running,Link to Azure Machine Learning studio,Link to Documentation
